In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

import numpy as np
import time
import copy

from MNIST_loader import loadDataWrapper

In [2]:
(x_train, y_train), (x_test, y_test) = loadDataWrapper()

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(No seed given, using 78565 instead!)
torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])
torch.Size([60000])
torch.Size([10000])


In [3]:
class datasetMaker(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return X.shape(0)
        
    def __getitem__(self, index):
        x = self.X[index,:,:]
        y = self.Y[index,:,:]
        return x, y


In [4]:

def train_epoch(epoch, optimizer, loss_function, model, train_loader):
    total_loss = 0
    model.train()
    for batch_idx, (data,label) in enumerate(train_loader):
        optimizer.zero_grad()
        predictions = model(data)
        loss_value = loss_function(predictions,label)
        loss_value.backward()
        optimizer.step()
        total_loss += loss_value.item()
    return total_loss / len(train_loader)

def validate_epoch(epoch, loss, model, val_loader):
    total_loss = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(val_loader):
            predictions = model(data)
            loss_value = loss(predictions, label)
            total_loss += loss_value.item()
    return total_loss / len(val_loader)


def a_proper_training(num_epoch, model, optimizer, loss_function, train_loader, val_loader):
    best_epoch = None
    best_model = None
    best_loss = float('inf')
    train_losses = list()
    val_losses = list()

    for epoch in range(num_epoch):
      
        start_time = time.time()  # Start time
        
        val_loss = validate_epoch(epoch, loss_function, model, val_loader)
        train_loss = train_epoch(epoch, optimizer, loss_function, model, train_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = copy.deepcopy(model)
            best_epoch = epoch
  
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Epoch {epoch + 1}/{round(num_epoch, 10)}: Train Loss={round(train_loss,10)} Val Loss={round(val_loss,10)} Test Loss={round(test_loss,10)} Elapsed_time = {round(elapsed_time/60,2)}minutes")
    
    return (best_model, best_epoch, train_losses, val_losses, test_losses, lrs)

In [5]:

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # 1 input channel (grayscale), 16 output channels, 3x3 convolution
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        
        # 16 input channels, 32 output channels, 3x3 convolution
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layer that takes the flattened features and outputs a tensor of size 9
        self.fc1 = nn.Linear(32 * 7 * 7, 9)  # 32 channels * 7x7 after pooling
        
    def forward(self, x):
        # Convolution layer 1 + ReLU activation + 2x2 Max Pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        
        # Convolution layer 2 + ReLU activation + 2x2 Max Pooling
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        # Flatten the feature maps into a 1D vector for the fully connected layer
        x = x.view(-1, 32 * 7 * 7)  # Flatten to batch size x 32*7*7
        
        # Fully connected layer
        x = self.fc1(x)
        
        return x

# Create an instance of the model
model = SimpleCNN()


NameError: name 'nn' is not defined

In [ ]:

trainset = datasetMaker(x_train, y_train)
testset = datasetMaker(x_test, y_test)

model = CNN()

# Now pass the scheduler to the training function
best_model, best_epoch, train_losses, val_losses, test_losses, lrs = a_proper_training(
    epochs, model, optimizer, criterion, train_loader, val_loader, test_loader, scheduler, verbose=verbose, patience=patience
)